### Importando biblíotecas necessárias

In [91]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Carregando a base de dados

In [92]:
dfBooks = pd.read_csv('archive/Books.csv')

C:\Users\Gabriel Lopes\AppData\Local\Temp\ipykernel_18276\2782996754.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  dfBooks = pd.read_csv('archive/Books.csv')


In [93]:
dfUsers = pd.read_csv('archive/Users.csv')

In [94]:
dfRatings = pd.read_csv('archive/Ratings.csv')

### Primeiras impressões dos dados

In [95]:
dfUsers.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [96]:
dfBooks.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [97]:
dfRatings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


### Renomeando colunas

In [98]:
dfBooks.rename(columns={'User-ID': 'user_id', 'Book-Author': 'author', 'Year-Of-Publication': 'year', 'Publisher': 'publisher', 'Book-Title': 'title'}, inplace=True)

In [99]:
dfRatings.rename(columns={'User-ID': 'user_id', 'Book-Rating': 'rating'}, inplace=True)

### Verificando a frequência de avalições dos usuários

In [100]:
count = dfRatings['user_id'].value_counts()

In [101]:
count

user_id
11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
116180        1
116166        1
116154        1
116137        1
276723        1
Name: count, Length: 105283, dtype: int64

Podemos perceber que há usuários que avaliam bem menos livros **e outros que mantem um hábito de leitura bem maior, logo avaliam mais livros. são esses que queremos usar para pegar de parametro para o modelo.**

In [102]:
dfRatings = dfRatings[dfRatings['user_id'].isin(count[count > 200].index)]

In [103]:
dfRatings.shape

(526356, 3)

**Metade do dataframe fo eliminado fazendo esse filtro dos usuários que avaliaram mais que 200 livros.**

In [104]:
rating_books = dfRatings.merge(dfBooks, on='ISBN')

In [105]:
rating_books.shape

(487671, 10)

In [106]:
num_rating = rating_books.groupby('title')['rating'].count().reset_index()

Contando a quantidade total de avaliações por cada titulo de livro.

In [107]:
num_rating.rename(columns={'rating': 'num_rating'}, inplace=True)

In [108]:
num_rating.shape

(160269, 2)

In [109]:
final_data = rating_books.merge(num_rating, on='title')

In [110]:
final_data.shape

(487671, 11)

In [111]:
final_data = final_data[final_data['num_rating'] >= 50]

Esta linha de comando seleciona apenas os livros que tem mais de 50 avaliações. **Isso filtrou cerca de 83% do dataset.** Caso não fosse aplicado esse filtro o modelo iria se perder dos livros mais importantes em meio a quantidade imensa com baixa quantidade de avaliação.

In [112]:
final_data.shape

(61853, 11)

In [113]:
final_data = final_data.drop_duplicates(['user_id', 'title'])

Dropando os dados duplicados.

In [114]:
final_data.shape

(59850, 11)

In [115]:
final_data.head(2)

,user_id,ISBN,rating,title,author,year,publisher,Image-URL-S,Image-URL-M,Image-URL-L,num_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,82
13,277427,0060930535,0,The Poisonwood Bible: A Novel,Barbara Kingsolver,1999,Perennial,http://images.amazon.com/images/P/0060930535.0...,http://images.amazon.com/images/P/0060930535.0...,http://images.amazon.com/images/P/0060930535.0...,133


In [116]:
final_data.drop(columns=['Image-URL-S', 'Image-URL-M'], inplace=True)

In [117]:
final_data.shape

(59850, 9)

In [118]:
final_data.rename(columns={'Image-URL-L': 'image_url'}, inplace=True)

In [119]:
final_data.head(2)

,user_id,ISBN,rating,title,author,year,publisher,image_url,num_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
13,277427,0060930535,0,The Poisonwood Bible: A Novel,Barbara Kingsolver,1999,Perennial,http://images.amazon.com/images/P/0060930535.0...,133


In [130]:
final_data.to_csv('archive/final_data.csv')

In [120]:
livros_mais_avaliados = final_data.sort_values(by='rating', ascending=False).head(5).sort_values(by='num_rating', ascending=False)
livros_mais_avaliados

,user_id,ISBN,rating,title,author,year,publisher,image_url,num_rating
290078,170513,0385504209,10,The Da Vinci Code,Dan Brown,2003,Doubleday,http://images.amazon.com/images/P/0385504209.0...,224
282068,166123,0312265840,10,Seven Up (A Stephanie Plum Novel),Janet Evanovich,2001,St. Martin's Press,http://images.amazon.com/images/P/0312265840.0...,111
282150,166123,0345339681,10,The Hobbit : The Enchanting Prelude to The Lor...,J.R.R. TOLKIEN,1986,Del Rey,http://images.amazon.com/images/P/0345339681.0...,83
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
282152,166123,0345339738,10,"The Return of the King (The Lord of the Rings,...",J.R.R. TOLKIEN,1986,Del Rey,http://images.amazon.com/images/P/0345339738.0...,80


In [121]:
livros_mais_avaliados['title'][0]

'Politically Correct Bedtime Stories: Modern Tales for Our Life and Times'

In [127]:
titles = []
urls = []
stars = []
num_ratings = []

for index, row in livros_mais_avaliados.iterrows():
    book_name = row['title']
    titles.append(book_name)
    url = row['image_url']
    urls.append(url)
    star = row['rating']
    star = star / 2
    stars.append(star)
    num_rating = row['num_rating']
    num_ratings.append(num_rating)

In [ ]:
from math import ceil
def star_print(star):
    no_star = '❌'
    star_moji = '⭐'
    star = ceil(star)
    if star == 0:
        print(no_star)
    else:
        print(star_moji * star)

In [139]:
star_print(star=stars[0])

⭐⭐⭐⭐⭐


In [ ]:
book_pivot = final_data.pivot_table(columns='user_id', index='title', values='rating').fillna(0)

In [ ]:
book_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,6323,6543,6563,6575,7158,7286,7346,7915,8067,8245,8681,8936,9856,10447,10819,11601,11676,11993,12538,12824,12982,13082,13273,13552,13850,14422,14521,15408,15418,15957,...,264317,264321,264637,265115,265313,265595,265889,266056,266226,266753,266865,266866,267635,268030,268032,268110,268330,268622,268932,269566,269719,269728,269890,270713,270820,271195,271284,271448,271705,273979,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


A função pivot_table do pandas é frequentemente usada em sistemas de recomendação para transformar dados de formato longo (onde cada linha representa uma interação entre um usuário e um item) em uma matriz de utilidade (ou matriz de interação), onde as linhas representam usuários, as colunas representam itens, e os valores representam as interações (como classificações, visualizações, etc.).

Essa matriz de utilidade é essencial para muitos algoritmos de recomendação, especialmente aqueles baseados em filtragem colaborativa, como o K-Nearest Neighbors (KNN) e a decomposição em valores singulares (SVD).

In [ ]:
from scipy.sparse import csr_matrix

A função csr_matrix do módulo scipy.sparse é usada para criar uma matriz esparsa no formato CSR (Compressed Sparse Row). Esse formato é eficiente em termos de memória e operações de matriz, especialmente quando a matriz contém muitos zeros, o que é comum em sistemas de recomendação.

Vantagens do Formato CSR

- Eficiência de Memória: Armazena apenas os elementos não-zero, economizando memória.

- Operações Rápidas: Permite operações eficientes de fatiamento e aritmética de matriz.

- Conversão Fácil: Pode ser facilmente convertida para outros formatos esparsos.

In [ ]:
book_pivot_normalized = book_pivot.sub(book_pivot.mean(axis=1), axis=0).fillna(0)
book_pivot_sparse = csr_matrix(book_pivot_normalized)

In [ ]:
book_pivot_sparse

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 658896 stored elements and shape (742, 888)>

In [ ]:
from sklearn.neighbors import NearestNeighbors

**Como Funciona o Algoritmo NearestNeighbors** <br><br>
O algoritmo NearestNeighbors pode ser configurado para usar diferentes métricas de distância e algoritmos de busca para encontrar os vizinhos mais próximos.

Métricas de Distância:

- euclidean: Distância Euclidiana.
- manhattan: Distância de Manhattan (ou L1).
- cosine: Distância do Cosseno.
- Outras métricas personalizadas.

Algoritmos de Busca:

- brute: Busca exaustiva (força bruta).
- kd_tree: Árvore KD (K-Dimensional).
- ball_tree: Árvore de Bolas.
- auto: Seleciona automaticamente o melhor algoritmo com base nos dados.
- Número de Vizinhos:

n_neighbors: Número de vizinhos mais próximos a serem encontrados.

In [ ]:
model = NearestNeighbors(algorithm='brute', metric='cosine')

In [ ]:
# Crie uma cópia para teste
train_data = book_pivot_sparse.copy()
test_data = book_pivot.copy()  # Mantém como DataFrame para facilitar acesso

# Oculte 20% das avaliações aleatoriamente
mask = np.random.rand(*book_pivot.shape) < 0.2  # 20% de teste
train_data[mask] = 0  # Zera as avaliações ocultas no treino
train_data = csr_matrix(train_data)  # Converte novamente para esparso

In [ ]:
model.fit(train_data)

NearestNeighbors(algorithm='brute', metric='cosine')

In [ ]:
distance, suggestions = model.kneighbors(book_pivot.iloc[237,:].values.reshape(1, -1), n_neighbors=6)


Análise da Função `model.kneighbors(...)`

Descrição: Esta função faz parte da classe `NearestNeighbors` do **scikit-learn** e é usada para identificar os vizinhos mais próximos de um ponto de consulta. O `model` é uma instância previamente treinada da classe `NearestNeighbors`.

Detalhes de `book_pivot.iloc[237, :]`:

- Contexto: `book_pivot` é um **DataFrame**, provavelmente uma matriz de utilidade onde:  
  - Linhas representam livros.  
  - Colunas representam usuários.  

- Seleção: `iloc[237, :]` seleciona a **linha 237** do DataFrame `book_pivot`. Essa linha contém as características do livro com índice **237**.

Transformação com `.values.reshape(1, -1)`:

- `.values`: Converte a linha selecionada em um **array NumPy**.  
- `.reshape(1, -1)`: Reorganiza o array em uma **matriz 2D** com:  
  - Uma única linha.  
  - Colunas suficientes para acomodar todos os elementos (-1 indica que o número de colunas é calculado automaticamente).  

  Essa transformação é necessária porque a função `kneighbors` requer que a entrada seja uma **matriz 2D**.

Parâmetro `n_neighbors=6`:

- Indica que a busca será feita para encontrar os **6 vizinhos mais próximos** do ponto de consulta.


In [ ]:
distance

array([[0.09947344, 0.53040077, 0.54777272, 0.59896281, 0.64389797,
        0.7766151 ]])

In [ ]:
suggestions

array([[237, 240, 238, 242, 241, 239]], dtype=int64)

**Medindo o desempenho do modelo**

Entrada: O código pega as características do livro na linha 237 do DataFrame book_pivot.

Processamento: Usa o modelo NearestNeighbors para encontrar os 6 livros mais próximos (vizinhos) ao livro na linha 237, com base nas características fornecidas.

Saída: A função kneighbors retorna duas coisas:

distance: Um array contendo as distâncias dos 6 livros mais próximos ao livro de consulta.

suggestions: Um array contendo os índices dos 6 livros mais próximos no DataFrame book_pivot.

In [ ]:
for i in range(len(suggestions)):
    print(book_pivot.index[suggestions[i]])

Index(['Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))',
       'Harry Potter and the Sorcerer's Stone (Book 1)',
       'Harry Potter and the Order of the Phoenix (Book 5)'],
      dtype='object', name='title')


In [ ]:
books_name = book_pivot.index

In [ ]:
books_name

Index(['1984', '1st to Die: A Novel', '2nd Chance', '4 Blondes',
       '84 Charing Cross Road', 'A Bend in the Road', 'A Case of Need',
       'A Child Called \It\": One Child's Courage to Survive"',
       'A Civil Action', 'A Cry In The Night',
       ...
       'Winter Solstice', 'Wish You Well', 'Without Remorse',
       'Wizard and Glass (The Dark Tower, Book 4)', 'Wuthering Heights',
       'Year of Wonders', 'You Belong To Me',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       'Zoya', '\O\" Is for Outlaw"'],
      dtype='object', name='title', length=742)


Exemplo prático:

Suponha que:

- `suggestions` seja `[237, 42, 15, 89, 300]`, ou seja, uma lista com 5 índices de livros que são "sugeridos" como similares (talvez encontrados pelo `kneighbors`).  
- `book_pivot.index` seja uma lista como `["O Senhor dos Anéis", "Harry Potter", "Código Da Vinci", ...]`, onde cada posição corresponde a um livro.

O loop fará o seguinte:

- Primeira iteração (`i = 0`):  
  `print(book_pivot.index[suggestions[0]])` imprime o livro na posição `suggestions[0]` (237). Se `book_pivot.index[237]` for "O Senhor dos Anéis", a saída será:  
  `O Senhor dos Anéis`

- Segunda iteração (`i = 1`):  
  `print(book_pivot.index[suggestions[1]])` imprime o livro na posição `suggestions[1]` (42). Se `book_pivot.index[42]` for "Harry Potter", a saída será:  
  `Harry Potter`

E assim por diante, até que todos os índices em `suggestions` sejam processados. No final, o código simplesmente lista os títulos ou identificadores dos livros que foram considerados "sugestões" ou "vizinhos mais próximos".


In [ ]:
import pickle

In [ ]:
# pickle.dump(books_name, open('artefatos/book_name.pkl', 'wb'))
# pickle.dump(model, open('artefatos/model.pkl', 'wb'))
# pickle.dump(final_data, open('artefatos/final_data.pkl', 'wb'))
# pickle.dump(book_pivot, open('artefatos/book_pivot.pkl', 'wb'))

In [ ]:
def evaluate_recommendations(book_title, model, train_data, test_data, k=6):
    # Encontre o ID do livro
    book_id = np.where(book_pivot.index == book_title)[0][0]
    
    # Obtenha recomendações
    distances, suggestions = model.kneighbors(train_data[book_id], n_neighbors=k)
    recommended_books = book_pivot.index[suggestions[0]]
    print(f"Recomendações para '{book_title}':", recommended_books.tolist())
    
    # Encontre livros relevantes (avaliações altas no conjunto completo)
    user_ratings = test_data.iloc[book_id, :].values  # Avaliações reais do livro
    true_likes = test_data.index[test_data.loc[:, test_data.columns[user_ratings > 7]].notna().any(axis=1)].tolist()
    true_likes = [book for book in true_likes if book != book_title]  # Exclui o próprio livro
    
    # Calcule acertos
    hits = len(set(recommended_books) & set(true_likes))
    precision = hits / k if k > 0 else 0
    recall = hits / len(true_likes) if true_likes else 0
    
    return precision, recall, true_likes

# Teste para um livro
book_title = "1984"
precision, recall, true_likes = evaluate_recommendations(book_title, model, train_data, test_data)
print(f"Precisão@{6}: {precision}, Recall@{6}: {recall}")
print(f"Livros relevantes reais: {true_likes}")

Recomendações para '1984': ['1984', 'Animal Farm', "Slaughterhouse Five or the Children's Crusade: A Duty Dance With Death", "The Handmaid's Tale", 'Brave New World', 'Roses Are Red (Alex Cross Novels)']
Precisão@6: 0.8333333333333334, Recall@6: 0.006747638326585695
Livros relevantes reais: ['1st to Die: A Novel', '2nd Chance', '4 Blondes', '84 Charing Cross Road', 'A Bend in the Road', 'A Case of Need', 'A Child Called \\It\\": One Child\'s Courage to Survive"', 'A Civil Action', 'A Cry In The Night', 'A Darkness More Than Night', 'A Day Late and a Dollar Short', 'A Fine Balance', 'A Great Deliverance', 'A Heartbreaking Work of Staggering Genius', 'A Is for Alibi (Kinsey Millhone Mysteries (Paperback))', 'A Lesson Before Dying (Vintage Contemporaries (Paperback))', 'A Man Named Dave: A Story of Triumph and Forgiveness', 'A Man in Full', 'A Map of the World', 'A Painted House', 'A Patchwork Planet', 'A Prayer for Owen Meany', 'A Thin Dark Line (Mysteries &amp; Horror)', "A Thousand Acr

In [ ]:
books_to_test = ["1984", "1st to Die: A Novel", "2nd Chance"]
precisions, recalls = [], []
for book in books_to_test:
    p, r, _ = evaluate_recommendations(book, model, train_data, book_pivot)
    precisions.append(p)
    recalls.append(r)
print(f"Média Precisão@6: {np.mean(precisions)}, Média Recall@6: {np.mean(recalls)}")

Recomendações para '1984': ['1984', 'Animal Farm', "Slaughterhouse Five or the Children's Crusade: A Duty Dance With Death", "The Handmaid's Tale", 'Brave New World', 'Roses Are Red (Alex Cross Novels)']
Recomendações para '1st to Die: A Novel': ['1st to Die: A Novel', 'Along Came a Spider (Alex Cross Novels)', 'The Sum of All Fears (Jack Ryan Novels)', 'Purity in Death', 'The Woman Next Door', 'Flesh and Blood']
Recomendações para '2nd Chance': ['2nd Chance', 'Flesh and Blood', 'Still Waters', 'Four Blind Mice', 'The Next Accident', 'The Murder Book']
Média Precisão@6: 0.8333333333333334, Média Recall@6: 0.006747638326585695


In [ ]:
book_id = np.where(book_pivot.index == 'Jacob Have I Loved')[0][0]
distance, suggestions = model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1, -1), n_neighbors=6)

In [ ]:
print(suggestions)

[[291 184  39 523 442 199]]


In [ ]:
def recomendation(book_title):
    book_id = np.where(book_pivot.index == book_title)[0][0]
    distance, suggestions = model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1, -1), n_neighbors=6)
    for i in range(len(suggestions)):
        books = (book_pivot.index[suggestions[i]])
        for j in books:
            print(j)

In [ ]:
final_data.to_csv('archive/final_data.csv')

In [ ]:
book_title = 'Jacob Have I Loved'
recomendation(book_title)

Jacob Have I Loved
Exclusive
Alaska
The Burden of Proof
Secrets
Fat Tuesday


In [ ]:
sum_rating = final_data[final_data['title'] == book_title]
sum_rating['rating'].sum()  

33